# Description
This notebook uses measurement error theory to recover the average causal effect between a treatment and outcome despite unmeasured confounding between the two variables.

In [274]:
import pandas as pd
import numpy as np
from scipy.special import expit
import statsmodels.api as sm

np.random.seed(1)
size = 1000000
verbose = True

U = np.random.binomial(1, 0.48, size)
if verbose:
    print(np.mean(U))

W = np.random.binomial(1, expit(1.3*U), size)
if verbose:
    print(np.mean(W))

Z = np.random.binomial(1, expit(0.5*U), size)
if verbose:
    print(np.mean(Z))

X = np.random.binomial(1, expit(0.8*U-0.4), size)
if verbose:
    print(np.mean(X))

Y = np.random.binomial(1, expit(1.2*U+X-0.8), size)
if verbose:
    print(np.mean(Y))

data = pd.DataFrame({"U": U, "W": W, "X": X, "Y": Y, "Z": Z})

0.480345
0.637066
0.558734
0.496623
0.556514


In [275]:
def compute_confidence_intervals(X, Y, Z, W, data, method_name, num_bootstraps=200, alpha=0.05):
    """
    Compute confidence intervals for backdoor adjustment via bootstrap
    
    Returns tuple (q_low, q_up) for the lower and upper quantiles of the confidence interval.
    """
    
    Ql = alpha/2
    Qu = 1 - alpha/2
    # two lists for the two indexes of output
    estimates0 = []
    estimates1 = []
    
    for i in range(num_bootstraps):
        
        # resample the data with replacement
        data_sampled = data.sample(len(data), replace=True)
        data_sampled.reset_index(drop=True, inplace=True)
        
        # add estimate from resampled data
        if method_name == "two proxy":
            output = two_proxy_effect_restoration(X, Y, Z, W, data_sampled)
            estimates0.append(output[0])
            estimates1.append(output[1])
        else:
            print("Invalid method")
            return

    # calculate the quantiles
    if method_name == "two proxy":
        quantiles = np.quantile(estimates0, q=[Ql, Qu])
        q_low0 = quantiles[0]
        q_up0 = quantiles[1]

        quantiles = np.quantile(estimates1, q=[Ql, Qu])
        q_low1 = quantiles[0]
        q_up1 = quantiles[1]
    
    return [(q_low0, q_up0), (q_low1, q_up1)]

# Effect Restoration
Code below attemps to recover p(X, Y, U) from p(X, Y, W) with additional information regarding p(W | U).

In [276]:
print("ground truth values")

groundTruth = []

for i in range(8):
    binString = format(i, '03b')

    data_subset = data[data["X"] == int(binString[0])]
    data_subset = data_subset[data_subset["Y"] == int(binString[1])]
    data_subset = data_subset[data_subset["U"] == int(binString[2])]

    groundTruth.append(len(data_subset)/size)

print(groundTruth)

print("p(X=1 | U)")
pX_U = []
data_subset = data[data["U"] == 0]
pX_U.append(np.mean(data_subset["X"]))
data_subset = data[data["U"] == 1]
pX_U.append(np.mean(data_subset["X"]))
print(pX_U)

ground truth values
[0.214789, 0.077593, 0.096087, 0.114908, 0.093969, 0.057135, 0.11481, 0.230709]
p(X=1 | U)
[0.40176463230412485, 0.5992442931642882]


In [277]:
print("given information: p(W | U)")

pWU = []

for i in range(2):
    binString = format(i, '01b')

    data_subset = data[data["U"] == int(binString[0])]

    # we only want to consider where rows where U=u
    subset_size = len(data_subset)
    data_subset = data_subset[data_subset["W"] == 0]

    pWU.append(len(data_subset)/subset_size)

    pWU.append(1 - (len(data_subset)/subset_size))

print(pWU)

given information: p(W | U)
[0.5007687792862572, 0.4992312207137428, 0.2138192340921629, 0.786180765907837]


In [278]:
def effect_restoration(data, X, Y, W, pWU):
    # recover the target law p(X, Y, U) from the observed data distribution
    # p(X, Y, W) from p(W | U), which is given information

    # find the observed data distribution p(X, Y, W) from observed data
    observedData = []

    for i in range(8):
        binString = format(i, '03b')

        data_subset = data[data[X] == int(binString[0])]
        data_subset = data_subset[data_subset[Y] == int(binString[1])]
        data_subset = data_subset[data_subset[W] == int(binString[2])]

        observedData.append(len(data_subset)/size)

    # calculate the inverse matrix of p(W | U) needed for effect restoration
    M = np.array([[pWU[0], pWU[2]], [pWU[1], pWU[3]]])
    I = np.linalg.inv(M)

    # find the estimates for the target law p(X, Y, U) using matrix multiplication
    estimates = []

    for i in range(0, 8, 2):
        pXYW = np.array([observedData[i], observedData[i+1]])
        result = np.matmul(I, pXYW)

        estimates.append(result[0])
        estimates.append(result[1])

    return estimates

In [279]:
print("target law:")
print(groundTruth)

# recover the data distribution p(X, Y, U)

print("estimates:")
recovered_distribution = effect_restoration(data, "X", "Y", "W", pWU)
print(recovered_distribution)

target law:
[0.214789, 0.077593, 0.096087, 0.114908, 0.093969, 0.057135, 0.11481, 0.230709]
estimates:
[0.21458512734709326, 0.07779687265290673, 0.09696551595474262, 0.11402948404525742, 0.09266457778754153, 0.05843942221245847, 0.11543977891062263, 0.2300792210893774]


In [280]:
print("ground truth values of p(X=1 | U):")
print(pX_U)

pX1U0 = recovered_distribution[4] + recovered_distribution[6]
pX1U1 = recovered_distribution[5] + recovered_distribution[7]
pU0 = recovered_distribution[0] + recovered_distribution[2] + recovered_distribution[4] + recovered_distribution[6]
pU1 = recovered_distribution[1] + recovered_distribution[3] + recovered_distribution[5] + recovered_distribution[7]

pX1_U0 = pX1U0/pU0
pX1_U1 = pX1U1/pU1

print(pX1_U0, pX1_U1)

ground truth values of p(X=1 | U):
[0.40176463230412485, 0.5992442931642882]
0.4004663799985839 0.6006487905606093


## Propensity Score Method
Propensity score method for recovering p(X=1 | U).

In [281]:
print("ground truth values of p(X=1 | U):")
print(pX_U)

# calculate the inverse matrix I
M = np.array([[pWU[0], pWU[2]], [pWU[1], pWU[3]]])
I = np.linalg.inv(M)

# store the values of I as a, b, c, d
a = I[0][0]
b = I[0][1]
c = I[1][0]
d = I[1][1]

# find the observed data distribution p(X, Y, W) from observed data
observedData = []

for i in range(8):
    binString = format(i, '03b')

    data_subset = data[data["X"] == int(binString[0])]
    data_subset = data_subset[data_subset["Y"] == int(binString[1])]
    data_subset = data_subset[data_subset["W"] == int(binString[2])]

    observedData.append(len(data_subset)/size)

pX1W0 = observedData[4] + observedData[6]
pX1W1 = observedData[5] + observedData[7]
pW0 = observedData[0] + observedData[2] + observedData[4] + observedData[6]
pW1 = observedData[1] + observedData[3] + observedData[5] + observedData[7]

pX1_U0 = (a*pX1W0 + b*pX1W1) / (a*pW0 + b*pW1)
print(pX1_U0)
pX1_U1 = (c*pX1W0 + d*pX1W1) / (c*pW0 + d*pW1)
print(pX1_U1)

ground truth values of p(X=1 | U):
[0.40176463230412485, 0.5992442931642882]
0.400466379998584
0.6006487905606094


# Two proxy effect restoration.
Code below attemps to recover p(X, Y, U) from p(X, Y, W, Z) without any other additional information.

To calculate eigenvalues and eigenvectors, refer to the following Python documentation: https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html.

In [283]:
def two_proxy_effect_restoration(X, Y, Z, W, data):
    # construct the matrix P(z, w) by using p(w=0 | x=0), p(z=0 | X=0), p(z=0, w=0 | X=0)
    data_subsetX0 = data[data[X] == 1]
    subset_size = len(data_subsetX0)

    data_subset = data_subsetX0[data_subsetX0[W] == 0]
    pW0_X0 = len(data_subset)/subset_size

    data_subset = data_subsetX0[data_subsetX0[Z] == 0]
    pZ0_X0 = len(data_subset)/subset_size

    data_subset = data_subset[data_subset[W] == 0]
    pZ0W0_X0 = len(data_subset)/subset_size

    P = np.array([[1, pW0_X0], [pZ0_X0, pZ0W0_X0]])

    # construct the matrix Q(z, w) by using p(y=0 | x=0), p(y=0, w=0 | x=0), p(y=0, z=0 | x=0), p(y=0, z=0, w=0 | x=0)
    data_subset = data_subsetX0[data_subsetX0[Y] == 0]
    pY0_X0 = len(data_subset)/subset_size

    data_subset = data_subset[data_subset[W] == 0]
    pY0W0_X0 = len(data_subset)/subset_size

    data_subset = data_subsetX0[data_subsetX0[Y] == 0]
    data_subset = data_subset[data_subset[Z] == 0]
    pY0Z0_X0 = len(data_subset)/subset_size

    data_subset = data_subset[data_subset[W] == 0]
    pY0Z0W0_X0 = len(data_subset)/subset_size

    Q = np.array([[pY0_X0, pY0W0_X0], [pY0Z0_X0, pY0Z0W0_X0]])

    P_inv = np.linalg.inv(P)

    # calculate the eigenvalues and eigenvectors of P(z, w)^{(-1)} * Q(z, w)
    eigenvalues, eigenvectors = np.linalg.eig(np.matmul(P_inv, Q))

    # print(eigenvalues)

    # find p(y=0 | x=0, u=0) and p(y=0 | x=0, u=1) to compare if the eigenvalues match up with
    # those values as they should in theory
    # data_subsetX0U0 = data_subsetX0[data_subsetX0["U"] == 0]
    # data_subset = data_subsetX0U0[data_subsetX0U0["Y"] == 0]
    # pY0_X0U0 = len(data_subset)/len(data_subsetX0U0)

    # data_subsetX0U1 = data_subsetX0[data_subsetX0["U"] == 1]
    # data_subset = data_subsetX0U1[data_subsetX0U1["Y"] == 0]
    # pY0_X0U1 = len(data_subset)/len(data_subsetX0U1)
    # print(pY0_X0U0, pY0_X0U1)
    # print()

    H_inv = np.linalg.inv(eigenvectors)

    alpha_1 = 1/H_inv[0][0]
    alpha_2 = 1/H_inv[1][0]

    return (alpha_1*H_inv[0][1], alpha_2*H_inv[1][1])

print(two_proxy_effect_restoration("X", "Y", "Z", "W", data))
print(compute_confidence_intervals("X", "Y", "Z", "W", data, "two proxy"))
print("ground truth:")
print(pWU[0], pWU[2])


(0.20924738762432915, 0.4907720502146312)
[(0.18657719263595357, 0.2250773111344551), (0.47040398083479246, 0.5175855897027777)]
ground truth:
0.5007687792862572 0.2138192340921629
